# Моделирование нагрузки велокурьеров (на примере склада №5018)

Велозона разбита на несколько сегментов: Кушелевская, Лесная, Блюхер, Кондратьевский-верх, Кондратьевский-низ, Революция (часовая зона)

Формальная очередь одна для всех зон

Неформальных очередей 6:
Кушелевская+Блюхер(опционально),
Лесная+Блюхер(опционально),
Кондратьевский-верх+Блюхер(опционально),
Кондратьевский-низ зона 30 мин,
Кондратьевский-низ часовая зона + Революция (часовая зона),
Экспрессы (все зоны)

Курьеры распределены по четырем группам:
Группа "Экспресс" (2-5 чел)
Группа "Ревал" (1-3 чел)
Группа "30-минут" (2-4 чел)
Группа "Общая" (5-25 чел)

Координация между группами отсутствует, в случае "подгорания" заказов, куртор или тот кто его замещает, говорит (т.е. это ручная работа) кому и куда ехать (в этом случае курьер берется из любой группы, как правило из общей).

Курьер смотрит через мобильное приложение "ВкусВилл курьер" на формальную очередь, в которой отображаются все заказы.
Одновременно он смотрит на список каналов в телеграм, где отображаются "неформальные" очереди. Курьер встает в очередь, ставя плюс в неформальную очередь.
После завершения набора, он ставит себе 👍 (лайк)

Группа "Общая" обслуживает одновременно три "неформальных" очереди: Кушелевская+Блюхер(опционально), Лесная+Блюхер(опционально), Кондратьевский-верх+Блюхер(опционально)
Группа "Экспресс" обслуживает одну "неформальную" очередь: Экспрессы
Группа "Ревал" обслуживает одну "неформальную" очередь: Кондратьевский-низ часовая зона + Революция (часовая зона)
Группа "30-минут" обслуживает одну "неформальную" очередь: Кондратьевский-низ зона 30 мин

Некоторое время назад была введена "блокировка" формальной очереди, речи идет о галочке, когда курьер должен взять первый заказ из формальной очереди для начала формирования маршрута. Ниже, исследование влияния этой блокировки на общую пропускную способность потока заказов.

Файл с нагрузкой, сгенерирован путем равномерного размазывания заказов на неделю. Используются рандомные значения, за неимением доступа к реальным данным.

Модель поведения курьера: Курьер набирается по максимуму что бы отвезти как можно больше заказов и не сделать "просрок", подобная модель поведения приводит к прижатию к верхней границе SLA для заказа.


In [4]:
import pandas as pd

orders_generator = pd.read_csv('./order-addresses.csv')
orders_generator.tail()

,order_address_id,order_address,order_address_capacity_per_week,order_address_average_bill
15,16,1-й Муринский проспект 19,1,1000
16,17,Лесной проспект 37к6,1,1000
17,18,Новолитовская улица 4,1,1000
18,19,Новолитовская улица 5,1,1000
19,20,Лесной проспект 67к1,1,1000


## методика вычисления среднего времени доставки и риска "просрока"

Среднее время доставки, интегральный показатель, который вычисляется по всем доставленным заказам. Риск просрока вычисляется исходя из близости времени доставки к SLA.

$max(w),max(h)$